In [ ]:
# 保存raw文件为csv文件，方便查看
import pandas as pd
from fy4a_projection_lookup import load_grid_data

lons_grid, lats_grid = load_grid_data('/mnt/md1/lxw/severe-convective-weather/FY4A/projection_table/table/FullMask_Grid_4000.raw', 4000)
lons_csv = '/mnt/md1/lxw/severe-convective-weather/FY4A/projection_table/table/FullMask_Grid_4000_lon.csv'  
lons_df = pd.DataFrame(lons_grid)
lons_df.to_csv(lons_csv, index=False, header=False)
lats_csv = '/mnt/md1/lxw/severe-convective-weather/FY4A/projection_table/table/FullMask_Grid_4000_lat.csv'  
lats_df = pd.DataFrame(lats_grid)
lats_df.to_csv(lats_csv, index=False, header=False)

In [ ]:
# 查表法得到的经纬度跟计算法得到的是一样的（至少在中国区是这样），结果如下：
# 纬度最大差异: 8.242295734817162e-13 度
# 经度最大差异: 1.6484591469634324e-12 度
# 纬度平均差异: 5.5089418567246105e-15 度
# 经度平均差异: 3.679484418824651e-15 度

import numpy as np
from fy4a_projection_calculator import linecolumn_to_latlon_calc
from fy4a_projection_lookup import linecolumn_to_latlon_lookup

line_range = [i for i in range(190, 190 + 730)]
column_range = [i for i in range(750, 750 + 1280)]
line, column = np.meshgrid(line_range, column_range, indexing='ij')
line = line.flatten()
column = column.flatten()

lat_calc, lon_calc = linecolumn_to_latlon_calc(line, column, resolution=4000)
lat_lookup, lon_lookup = linecolumn_to_latlon_lookup(line, column, resolution=4000)
diff_lat = lat_calc - lat_lookup
diff_lon = lon_calc - lon_lookup
print(f"纬度最大差异: {np.max(np.abs(diff_lat))} 度")
print(f"经度最大差异: {np.max(np.abs(diff_lon))} 度")
print(f"纬度平均差异: {np.mean(np.abs(diff_lat))} 度")
print(f"经度平均差异: {np.mean(np.abs(diff_lon))} 度")

print(lat_calc[100000:100010])
print(lon_calc[100000:100010])
print(lat_lookup[100000:100010])
print(lon_lookup[100000:100010])

In [ ]:
import time
import numpy as np
from fy4a_projection_calculator import latlon_to_linecolumn_calc
from fy4a_projection_lookup import latlon_to_linecolumn_lookup, linecolumn_to_latlon_lookup

# 生成中国区经纬度测试点
lats = np.linspace(18.0, 53.0, 50)
lons = np.linspace(73.0, 135.0, 50)
lat_grid, lon_grid = np.meshgrid(lats, lons, indexing='ij')
lat_test = lat_grid.flatten()
lon_test = lon_grid.flatten()

# 计算法
start_time_calc = time.time()
line_calc, col_calc = latlon_to_linecolumn_calc(lat_test, lon_test, resolution=4000)
end_time_calc = time.time()
lat_calc_back, lon_calc_back = linecolumn_to_latlon_lookup(line_calc, col_calc, resolution=4000)

# 查找法：
start_time_lookup = time.time()
line_lookup, col_lookup = latlon_to_linecolumn_lookup(lat_test, lon_test, resolution=4000)
end_time_lookup = time.time()
lat_lookup_back, lon_lookup_back = linecolumn_to_latlon_lookup(line_lookup, col_lookup, resolution=4000)

# 计算差异
diff_lat_calc = lat_test - lat_calc_back
diff_lon_calc = lon_test - lon_calc_back
diff_lat_lookup = lat_test - lat_lookup_back  
diff_lon_lookup = lon_test - lon_lookup_back

print("\n=== 计算法精度 ===")
print(f"纬度最大差异: {np.max(np.abs(diff_lat_calc))} 度")
print(f"经度最大差异: {np.max(np.abs(diff_lon_calc))} 度") 
print(f"纬度平均差异: {np.mean(np.abs(diff_lat_calc))} 度")
print(f"经度平均差异: {np.mean(np.abs(diff_lon_calc))} 度")

print("\n=== 查找法精度 ===")
print(f"纬度最大差异: {np.max(np.abs(diff_lat_lookup))} 度")
print(f"经度最大差异: {np.max(np.abs(diff_lon_lookup))} 度")
print(f"纬度平均差异: {np.mean(np.abs(diff_lat_lookup))} 度") 
print(f"经度平均差异: {np.mean(np.abs(diff_lon_lookup))} 度")

# 检查两种方法行列号的一致性
print("\n=== 跟计算法行列号差异 ===")
line_diff = np.abs(line_calc - line_lookup)
col_diff = np.abs(col_calc - col_lookup)
print(f"行号最大差异: {np.max(line_diff)} 像素")
print(f"列号最大差异: {np.max(col_diff)} 像素")
print(f"行号平均差异: {np.mean(line_diff)} 像素")
print(f"列号平均差异: {np.mean(col_diff)} 像素")

print(f"\n计算法用时: {end_time_calc - start_time_calc} 秒")
print(f"查找法用时: {end_time_lookup - start_time_lookup} 秒")

print(lat_test[10:70])
print(lon_test[10:70])


# 计算法：
# 用时: 0.0003459453582763672 秒
# === 计算法精度 ===
# 纬度最大差异: 0.040279105670151694 度
# 经度最大差异: 0.06459290120308481 度
# 纬度平均差异: 0.013262601662697543 度
# 经度平均差异: 0.013431598905840388 度

# 球面距离查找法：根据Haversine公式
# 用时: 635.6044342517853 秒
# === 查找法精度 ===
# 纬度最大差异: 0.040279105670151694 度
# 经度最大差异: 0.04525761803634509 度
# 纬度平均差异: 0.013109713642743737 度
# 经度平均差异: 0.01273008010061746 度
# === 跟计算法行列号差异 ===
# 行号最大差异: 1 像素
# 列号最大差异: 1 像素
# 行号平均差异: 0.0244 像素
# 列号平均差异: 0.0624 像素


# 先纬度后经度查找法：假设同一行纬度一样（虽然不是），先找最近的行，再找最近的列
# 用时: 98.63113260269165 秒
# === 查找法精度 ===
# 纬度最大差异: 3.4214714198021525 度
# 经度最大差异: 0.0375408567669524 度
# 纬度平均差异: 0.8347600909153123 度
# 经度平均差异: 0.01280043253317807 度
# === 跟计算法行列号差异 ===
# 行号最大差异: 66 像素
# 列号最大差异: 24 像素
# 行号平均差异: 16.7184 像素
# 列号平均差异: 3.5572 像素